In [1]:
# Make some imports

import numpy as np
import pandas as pd
from pandas import DataFrame
from collections import defaultdict
import pickle
import sklearn as sk
from sklearn.model_selection import train_test_split

In [2]:
world_happiness = pd.read_csv ('../data/world-happiness-report.csv')
world_happiness_shape = world_happiness.shape
print(world_happiness)

     Country name  year  Life Ladder  Log GDP per capita  Social support  \
0     Afghanistan  2008        3.724               7.370           0.451   
1     Afghanistan  2009        4.402               7.540           0.552   
2     Afghanistan  2010        4.758               7.647           0.539   
3     Afghanistan  2011        3.832               7.620           0.521   
4     Afghanistan  2012        3.783               7.705           0.521   
...           ...   ...          ...                 ...             ...   
1944     Zimbabwe  2016        3.735               7.984           0.768   
1945     Zimbabwe  2017        3.638               8.016           0.754   
1946     Zimbabwe  2018        3.616               8.049           0.775   
1947     Zimbabwe  2019        2.694               7.950           0.759   
1948     Zimbabwe  2020        3.160               7.829           0.717   

      Healthy life expectancy at birth  Freedom to make life choices  \
0              

In [3]:
# there are 373 null values in world_happiness and we currently find which country the null value belongs to 
# and replace it with the mean of the column of that country.

# 1. create a dictionary with keys of countries and values of their data:

world_happiness_dict = defaultdict()
for row in range(world_happiness_shape[0]):
    value = world_happiness.loc[row].tolist()
    keys = world_happiness_dict.keys()
    current_country = world_happiness.loc[row][0]
    if (current_country in world_happiness_dict.keys()):
        world_happiness_dict[current_country].append(value[1:])
    else:
        world_happiness_dict[current_country] = [value[1:]]

In [4]:
# We delte the countries with only one row of values, for which we cannot find the current column mean. 
country_with_one_row_data = []
for country in world_happiness_dict.keys():
    if len(world_happiness_dict.get(country)) == 1:
        country_with_one_row_data.append(country)
print("the countries with only one row of data are:", country_with_one_row_data)
print("We delete these 5")
for country in country_with_one_row_data:
    world_happiness_dict.pop(country, None)

the countries with only one row of data are: ['Cuba', 'Guyana', 'Maldives', 'Oman', 'Suriname']
We delete these 5


In [5]:
# 2. find all "nan" value and replace it with the current mean of the column of the country:
# traverse all the keys in the dictionary and for each country, 
filled_data = defaultdict()
for country in world_happiness_dict.keys():
    df = pd.DataFrame(world_happiness_dict.get(country))
    filled_df = df.fillna(df.mean())
    if filled_df.isnull().values.any():
        filled_df = df.fillna(0)
    filled_data[country] = filled_df
            

In [6]:
#3. construct a dataframe to extract 80% of the data as the training data and 20% as the testing data. 
# There are 166 countries within the dataset
# Within the dataset, there are in total 1949 data entries

print(len(filled_data.keys()))
total = 0


training_set = []
testing_set = []

for country in filled_data.keys():
    total_data = filled_data.get(country)
    train, test = train_test_split(total_data, test_size=0.2, random_state=50, shuffle=True)

    training_set.append(train)
    testing_set.append(test)

training_data = pd.concat(training_set)
testing_data = pd.concat(testing_set)

print("Training data dimensions: ", training_data.shape)
print("Testing data dimensions: ", testing_data.shape)

161
Training data dimensions:  (1515, 10)
Testing data dimensions:  (429, 10)


In [7]:
print(training_data.isnull().values.any())
print(testing_data.isnull().values.any())

False
False


In [8]:
training_data.to_pickle("./train.pkl")
testing_data.to_pickle("./test.pkl")